In [14]:
import credentials # Api key is stored in this file, remove to avoid errors if you clone from github

import pvdeg
import pvlib
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from scipy.linalg import cholesky

First get weather data and metadata for a desired location (latitude and logitude)

In [15]:
# change to desired values (currently Miami)
latitude = 25.783388
longitude = -80.189029

API_KEY = credentials.API_KEY # my personal NREL api key
email ='tobin.ford@nrel.gov' # replace these values with your appropriate information and remove or comment out first line of first block (import credentials)

# reads NSRDB data 
weather_df, meta = pvlib.iotools.get_psm3(latitude, longitude, API_KEY, email, names='2019', map_variables=True)

c:\ProgramData\anaconda3\lib\site-packages\pvlib\iotools\psm3.py:185: pvlibDeprecationWarning: The ``get_psm3`` function will default to leap_day=True starting in pvlib 0.11.0. Specify leap_day=True to enable this behavior now, or specify leap_day=False to hide this warning.
  warnings.warn(


User has 3 parameters for initial implementation: See Kempe's "Deg Miami" tab in excel<br>



activation energy, Ea <br>
irradiance relation, x<br>
ln(R0)<br>

|           |   Ea   |   x   | ln(R0) |
|:---------:|:-----:|:----:|:-------:|
|   **Ea**  |   1   |   a  |   b     |
|   **x**   |   a   |   1  |   c     |
| **ln(R0)**|   b   |   c  |   1     |

Notice symmetry across diagonal <br>

In [118]:
# USER ENTERED VALUES
# Correlation Coefficients
Ea_X = 0.0269
Ea_lnR0 = -0.9995 
X_lnR0 = -0.0400

# Activation Energy
mean_Ea = 62.08 # average
sd_Ea = 7.3858 # standard deviation

# Irradiance relation
mean_X = 0.0341 # average
sd_X = 0.0992757 # standard deviation

# ln(R0)
mean_lnR0 = 13.7223084 
sd_lnR0 = 2.47334772

# number of iterations
n = 20000


In [129]:
# notice symmetry of matrix
A = np.array([[1,   Ea_X,   Ea_lnR0],
              [Ea_X,    1,   X_lnR0],
              [Ea_lnR0, X_lnR0,   1]])

# conceptually similar to the square root of a matrix
A_decomp = cholesky(A, lower=True) # does it need to be upper, either way lnR0 values are very wrong

# generating standard distribution of points for each
ea, x, lnR0 = np.random.normal(mean_Ea, sd_Ea, 20_000), np.random.normal(mean_X, sd_X, 20_000), np.random.normal(mean_lnR0, sd_lnR0, 20_000)

# transpose built in so I don't have to do it with another function
samples_matrix = np.array([ea, x, lnR0])

data = {
    'ea': ea,
    'x': x,
    'lnR0': lnR0
}

uncorrelated_df = pd.DataFrame(data)

(3, 20000)

In [130]:
data = {
    'ea': ea,
    'x': x,
    'lnR0': lnR0
}

uncorrelated_df = pd.DataFrame(data)

In [156]:
print(ea.min())
print(ea.max())
print()

print(x.min())
print(x.max())
print()

print(lnR0.min())
print(lnR0.max())
print()

[61.08364242 67.41366698 61.6170852  ... 61.69540837 63.92443513
 73.2215795 ]
[ 0.22582344  0.01695655  0.15120942 ... -0.09589325  0.09273276
 -0.0973393 ]
[13.6018775  13.10790037 12.78734999 ... 16.08331126 14.42221943
 11.11978797]
35.35122443782616
92.08061122639748

-0.3247726302484897
0.4410425300301098

3.0889064251239926
23.3330823652636



In [114]:
# correlated stats pre-input to function

# not entirely sure what I should do here,
# do i multiply the transpose by the lower cholesky to get correlated input data for the equation on the spreadsheet
# using MonteCarloEaLnRoX

# this gives us correlated input samples
# looks like what kempe's code was doing in "MonteCarloEaLnRoX" but it is hard to tell what's happening in his code
correlated_samples = np.matmul(A_decomp, samples_matrix)

correlated_df = pd.DataFrame(correlated_samples.T, columns=['ea', 'x', 'lnR0'])

sol_pos = pvdeg.spectral.solar_position(weather_df, meta)
poa_irradiance = pvdeg.spectral.poa_irradiance(weather_df, meta)
temp_mod = pvdeg.temperature.module(weather_df=weather_df, meta=meta, poa=poa_irradiance, conf='open_rack_glass_polymer')
# we only care about global irradiance in this case


In [115]:
print(correlated_df)

              ea         x       lnR0
0      65.604340  1.783860 -65.221264
1      66.963367  1.894129 -66.430842
2      63.720007  1.790020 -63.266888
3      46.086021  1.320460 -45.725141
4      73.748773  1.750173 -73.383858
...          ...       ...        ...
19995  65.616039  1.838830 -65.162291
19996  50.571004  1.280881 -50.173422
19997  47.826794  1.412995 -47.433248
19998  77.359359  2.016745 -76.921411
19999  59.020193  1.623082 -58.447982

[20000 rows x 3 columns]


In [116]:
# attempting to compare my function input values to kempe's from macro
# Ea min:30.187
# Ea max:89.94

# lnR0 min: 4.477
# lnR0 max: 24.31

# X min:-0.387
# X max: 0.415

# uncorrelated samples of mine
'''
ea
34.50720101130711
95.14753086962446

lnR0
4.258833206368452
23.659585717113735

x
-0.349302212696885
0.4300323062342357
'''
# these are very similar to kempes that he was feeding to the arrhenius equation in the macro, not similar to my correlated variables at all?
# what happens if i feed my uncorrelated variables to the function


print(correlated_df['ea'].min())
print(correlated_df['ea'].max())

print(correlated_df['lnR0'].min())
print(correlated_df['lnR0'].max())

print(correlated_df['x'].min())
print(correlated_df['x'].max())

# can conclude there is something weird happening with my input data. 

26.41720111791205
92.14155728073086
-91.69088006622732
-26.01085300481374
0.6411146460608543
2.5393324260528165


In [151]:
# i know parameters I wrong but leaving for testing sake
def mikeArrhenius(weather_df, samples):

    #['Ea', 'X', 'lnR0']
    #d = [(np.exp(z) * np.exp((x / 8.31446261815324E-03) / (273.15 + weather_df['temp_air'])) * (weather_df['ghi'] / 1000 ** y)).mean() for x, y, z in zip(samples['ea'], samples['x'], np.log(samples['lnR0']))]
    
    d = [(np.exp(z) * np.exp((x / 8.31446261815324E-03) / (273.15 + temp_mod)) * (poa_irradiance['poa_global'] / 1000 ** y)).mean() for x, y, z in zip(samples['ea'], samples['x'], np.log(samples['lnR0']))]
    # tried using cell temp
    # tried using poa

    return pd.DataFrame(d)

In [152]:
temp2 = mikeArrhenius(weather_df=weather_df, samples=uncorrelated_df) 

# trying with uncorrelated variables

# difference is probably something about how I am passing values to the functions,
# the original function takes data as arguments but the second just pulls them from globalscope, this could cause undesired behavior

In [154]:
temp2.head(24) 
# each row shows the degredation rate for that given monte carlo trial iteration
# data frame indexes are integers corresponding to each trial

,0
0,2.545516e+13
1,1.317913e+15
2,4.963486e+13
3,7.230700e+13
4,1.833524e+12
5,3.247497e+16
6,1.445010e+14
7,6.303645e+13
8,4.152183e+13
9,9.094371e+13
